<a href="https://colab.research.google.com/github/dinhluan14/Classification-of-social-media-text-in-Vietnamese/blob/main/LSTM_UIT_VSMEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://drive.google.com/file/d/1ch_F1fRqgBuT5bLLk6CbJoHO1TpvEEnQ/view?usp=sharing

In [ ]:
https://drive.google.com/file/d/1svJAKgX2W6CzLk5wWt4A5oKnn25yNha9/view?usp=sharing
https://drive.google.com/file/d/1IqxO0hhfVyNbDWkbbwyuz9NGM6G_md_R/view?usp=sharing

In [ ]:
https://drive.google.com/file/d/1IqxO0hhfVyNbDWkbbwyuz9NGM6G_md_R/view?usp=sharing
https://drive.google.com/file/d/1svJAKgX2W6CzLk5wWt4A5oKnn25yNha9/view?usp=sharing
https://drive.google.com/file/d/1ch_F1fRqgBuT5bLLk6CbJoHO1TpvEEnQ/view?usp=sharing

#Load DataSet


In [ ]:
!gdown --id 1ch_F1fRqgBuT5bLLk6CbJoHO1TpvEEnQ
!gdown --id 1svJAKgX2W6CzLk5wWt4A5oKnn25yNha9
!gdown --id 1IqxO0hhfVyNbDWkbbwyuz9NGM6G_md_R

Downloading...
From: https://drive.google.com/uc?id=1ch_F1fRqgBuT5bLLk6CbJoHO1TpvEEnQ
To: /content/valid_nor_811.xlsx
100% 38.0k/38.0k [00:00<00:00, 14.1MB/s]


In [ ]:
xinchaoao